<a href="https://colab.research.google.com/github/isshiki/neural-network-by-code/blob/main/01-forward-prop/nn_from_scratch_without_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2021-2022 Digital Advantage - Deep Insider.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

連載『ニューラルネットワーク入門』
# 「コードで必ず分かるニューラルネットワーク（DNN）の逆伝播」


<table valign="middle">
  <td>
    <a target="_blank" href="https://atmarkit.itmedia.co.jp/ait/articles/2202/09/news027.html"> <img src="https://re.deepinsider.jp/img/ml-logo/manabu.svg"/>Deep Insiderで記事を読む</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/isshiki/neural-network-by-code/blob/main/01-forward-prop/nn_from_scratch_without_numpy.ipynb"> <img src="https://re.deepinsider.jp/img/ml-logo/gcolab.svg" />Google Colabで実行する</a>
  </td>
  <td>
    <a target="_blank" href="https://studiolab.sagemaker.aws/import/github/isshiki/neural-network-by-code/tree/main/01-forward-prop/nn_from_scratch_without_numpy.ipynb"> <img src="https://re.deepinsider.jp/img/ml-logo/astudiolab.svg" />AWS  SageMaker Studio Labで実行する</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/isshiki/neural-network-by-code/blob/main/01-forward-prop/nn_from_scratch_without_numpy.ipynb"> <img src="https://re.deepinsider.jp/img/ml-logo/github.svg" />GitHubでソースコードを見る</a>
  </td>
</table>

## ■本ノートブックの目的

ニューラルネットワーク（以下、ニューラルネット）の仕組みを、数学理論からではなく**Pythonコードから学ぶ**ことを狙っています。「難しい高校以降の数学は苦手だけど、コードなら読めるぜ！」という方にピッタリです。

### ●本ノートブックの特徴

- 線形代数（linear algebra、行列演算）を使いません。つまり、NumPyを使いません。
- 基本的に掛け算や足し算などの中学までの数学のみで、ニューラルネットのロジックをコーディングしていきます。
- ※微分の導関数は、そのままコードとして記載することで、微分の計算は取り上げません。

![図1　線形代数を使わないことによるメリット](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/01-forward-prop/images/01.png)

## ■ニューラルネットワークの図

基本的なニューラルネット（この例では、入力層：2、隠れ層：3、出力層：1）の図を確認しておきます。

![図2　ニューラルネットワークの図（左：横描き、右：縦描き）](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/01-forward-prop/images/02.png)

## ■訓練（学習）処理全体の実装

深層「学習」のメインは、ニューラルネットの「訓練」処理ですよね。ここから書いていきます。

In [ ]:
# 取りあえず仮で、空の関数を定義して、コードが実行できるようにしておく
def forward_prop(cache_mode=False):
    " 順伝播を行う関数。"
    return None, None, None

y_true = 1.0  # 正解値
def back_prop(y_true, cached_outs, cached_sums):
    " 逆伝播を行う関数。"
    return None, None

LEARNING_RATE = 0.1 # 学習率（lr）
def update_params(grads_w, grads_b, lr=0.1):
    " パラメーター（重みとバイアス）を更新する関数。"
    return None, None

# ---ここまでは仮の実装。ここからが必要な実装---

# 訓練処理
y_pred, cached_outs, cached_sums = forward_prop(cache_mode=True)  # （1）
grads_w, grads_b = back_prop(y_true, cached_outs, cached_sums)  # （2）
weights, biases = update_params(grads_w, grads_b, LEARNING_RATE)  # （3）

print(f'予測値：{y_pred}')  # 予測値： None
print(f'正解値：{y_true}')  # 正解値： 1.0

ニューラルネットの訓練に必要なことは、以下の3つだけ。

1. **順伝播：** `forward_prop()◎`数として実装
2. **逆伝播：** `back_prop()`関数として実装。損失（予測と正解の誤差）の計算はここで行う
3. **パラメーター（重みとバイアス）の更新：** `update_params()`関数として実装。これによりモデルが**最適化**される


![図3　訓練（学習）処理を示したニューラルネットワーク図](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/01-forward-prop/images/03.png)

##■モデルの定義と、仮の訓練データ

入力層のノードが2個、隠れ層のノードが3個、出力層のノードが1個のモデル（`model`変数）を定義しましょう。

In [ ]:
# ニューラルネットワークは3層構成
layers = [
    2,  # 入力層の入力（特徴量）の数
    3,  # 隠れ層1のノード（ニューロン）の数
    1]  # 出力層のノードの数

# 重みとバイアスの初期値
weights = [
    [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], # 入力層→隠れ層1
    [[0.0, 0.0, 0.0]] # 隠れ層1→出力層
]
biases = [
    [0.0, 0.0, 0.0],  # 隠れ層1
    [0.0]  # 出力層
]

# モデルを定義
model = (layers, weights, biases)

# 仮の訓練データ（1件分）を準備
x = [0.05, 0.1]  # x_1とx_2の2つの特徴量

## ■ステップ1. 順伝播の実装

### ●1つのノードにおける順伝播の処理

ニューラルネットの最小単位である「1つのノード」における順伝播の処理をコーディングしましょう。

In [ ]:
# 取りあえず仮で、空の関数を定義して、コードが実行できるようにしておく
def summation(x,weights, bias):
    " 重み付き線形和の関数。"
    return 0.0

def sigmoid(x):
    " シグモイド関数。"
    return 0.0

def identity(x):
    " 恒等関数。"
    return 0.0


w = [0.0, 0.0]  # 重み（仮の値）
b = 0.0  # バイアス（仮の値）

next_x = x  # 訓練データをノードへの入力に使う

# ---ここまでは仮の実装。ここからが必要な実装---

# 1つのノードの処理（1）： 重み付き線形和
node_sum = summation(next_x, w, b)

# 1つのノードの処理（2）： 活性化関数
is_hidden_layer = True
if is_hidden_layer:
    # 隠れ層（シグモイド関数）
    node_out = sigmoid(node_sum)
else:
    # 出力層（恒等関数）
    node_out = identity(node_sum)

1つのノードの順伝播処理に必要なことは、以下の2つの数学関数だけ。

1. **重み付き線形和の関数：** `summation()`関数として実装
2. **活性化関数：** ここでは`sigmoid()`関数や`identity()`関数として実装

![図4　1つのニューロンにおける順伝播の処理を示した図](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/01-forward-prop/images/04.png)

### ●重み付き線形和

重み付き線形和（weighted linear summation、以下では「線形和」と表記）とは、とは、あるノードへの複数の入力（$x_1$、$x_2$など）に、それぞれの重み（$w_1$、$w_2$など）を掛けて足し合わせて、最後にバイアス（$b$）を足した値です（上の図の左）。

In [ ]:
def summation(x, weights, bias):
    """
    重み付き線形和の関数。
    ※1データ分、つまりxとweightsは「一次元リスト」という前提。
    - 引数：
    x： 入力データをリスト値（各要素はfloat値）で指定する。
    weights： 重みをリスト値（各要素はfloat値）で指定する。
    bias： バイアスをfloat値で指定する。
    - 戻り値：
    線形和の計算結果をfloat値で返す。
    """
    linear_sum = 0.0
    for x_i, w_i in zip(x, weights):
        linear_sum += x_i * w_i  # iは「番号」（数学は基本的に1スタート）
        # print(f'x_i({x_i})×w_i({w_i})＋', end='')
    linear_sum += bias
    # print(f'b({bias})', end='')
    return linear_sum

# 線形代数を使う場合のコード例：
# linear_sum = np.dot(x, weights) + bias

ついで、次回の逆伝播（の中で使う偏微分）で必要となる線形和（linear **sum**mation）の偏導関数（partial **der**ivative function）を実装しておきます。

In [ ]:
def sum_der(x, weights, bias, with_respect_to='w'):
    """
    重み付き線形和の関数の偏導関数。
    ※1データ分、つまりxとweightsは「一次元リスト」という前提。
    - 引数：
    x： 入力データをリスト値で指定する。
    weights：  重みをリスト値で指定する。
    bias: バイアスをfloat値で指定する。
    with_respect_to: 何に関して偏微分するかを指定する。
        'w'＝ 重み、'b'＝ バイアス、'x'＝ 入力。
    - 戻り値：
    with_respect_toが'w'や'x'の場合はリスト値で、'b'の場合はfloat値で
        線形和の偏微分の計算結果（偏微分係数）を返す。
    """    
    if with_respect_to == 'w':
        return x  # 線形和uを各重みw_iで偏微分するとx_iになる（iはノード番号）
    elif with_respect_to == 'b':
        return 1.0  # 線形和uをバイアスbで偏微分すると1になる
    elif with_respect_to == 'x':
        return weights  # 線形和uを各入力x_iで偏微分するとw_iになる

### ●活性化関数：シグモイド関数

隠れ層では、最も基礎的なシグモイド関数（Sigmoid function）を固定的に使うことにします。導関数も実装しておきます。

In [ ]:
import math

def sigmoid(x):
    """
    シグモイド関数。
    - 引数：
    x： 入力データをfloat値で指定する。
    - 戻り値：
    シグモイド関数の計算結果をfloat値で返す。
    """
    return 1.0 / (1.0 + math.exp(-x))

# 線形代数の場合はmathをnpに変える（事前にimport numpy as np）

In [ ]:
def sigmoid_der(x):
    """
    シグモイド関数の（偏）導関数。
    - 引数：
    x： 入力データをfloat値で指定する。
    - 戻り値：
    シグモイド関数の（偏）微分の計算結果（微分係数）をfloat値で返す。
    """
    output = sigmoid(x)
    return output * (1.0 - output)

### ●活性化関数：恒等関数

出力層では、回帰問題をイメージして、そのままの値を出力する活性化関数である恒等関数（Identity function）を使用します。導関数も実装しておきます。

In [ ]:
def identity(x):
    """
    恒等関数の関数。
    - 引数：
    x： 入力データをfloat値で指定する。
    - 戻り値：
    恒等関数の計算結果（そのまま）をfloat値で返す。
    """
    return x

In [ ]:
def identity_der(x):
    """
    恒等関数の（偏）導関数。
    - 引数：
    x： 入力データをfloat値で指定する。
    - 戻り値：
    恒等関数の（偏）微分の計算結果（微分係数）をfloat値で返す。
    """
    return 1.0

### ●順伝播の処理全体の実装

ニューラルネットには、層があり、その中に複数のノードが存在するという構造です。  従って、

- 各層を1つずつ処理する`for`ループと、  
  - 層の中のノードを1つずつ処理する`for`ループの2段階構造が必要で、
    - その中に「1つのノードにおける順伝播の処理」

を記述すればよいわけです。

In [ ]:
def forward_prop(layers, weights, biases, x, cache_mode=False):
    """
    順伝播を行う関数。
    - 引数：
    (layers, weights, biases)： モデルを指定する。
    x： 入力データを指定する。
    cache_mode： 予測時はFalse、訓練時はTrueにする。これにより戻り値が変わる。
    - 戻り値：
    cache_modeがFalse時は予測値のみを返す。True時は、予測値だけでなく、
        キャッシュに記録済みの線形和（Σ）値と、活性化関数の出力値も返す。
    """

    cached_sums = []  # 記録した全ノードの線形和（Σ）の値
    cached_outs = []  # 記録した全ノードの活性化関数の出力値

    # まずは、入力層を順伝播する
    # print(f'■第1層（入力層）-全て（{len(x)}個）の特徴量：')
    # print(f'　●入力データ： ', end='')
    cached_outs.append(x)  # 何も処理せずに出力値を記録
    # print(f'何もしない＝out({x})')
    next_x = x  # 現在の層の出力（x）＝次の層への入力（next_x）

    # 次に、隠れ層や出力層を順伝播する
    SKIP_INPUT_LAYER = 1
    for layer_i, layer in enumerate(layers):  # 各層を処理
        if layer_i == 0:
            continue  # 入力層は上で処理済み

        # 各レイヤーのノードごとに処理を行う
        sums = []
        outs = []
        for node_i in range(layer):  # 層の中の各ノードを処理
            # print(f'■第{layer_i+1}層-第{node_i+1}ノード：')

            # ノードごとの重みとバイアスを取得
            w = weights[layer_i-SKIP_INPUT_LAYER][node_i]
            b = biases[layer_i-SKIP_INPUT_LAYER][node_i]

            # 1つのノードの処理（1）： 重み付き線形和
            # print(f'　●重み付き線形和： ', end='')
            node_sum = summation(next_x, w, b)
            # print(f'＝sum({node_sum})')

            # 1つのノードの処理（2）： 活性化関数
            if layer_i < len(layers)-1:  # -1は出力層以外の意味
                # 隠れ層（シグモイド関数）
                # print(f'　●活性化関数（隠れ層はシグモイド関数）： ', end='')
                node_out = sigmoid(node_sum)
                # print(f'sigmoid({node_sum})＝out({node_out})')
            else:
                # 出力層（恒等関数）
                # print(f'　●活性化関数（出力層は恒等関数）： ', end='')
                node_out = identity(node_sum)
                # print(f'identity({node_sum})＝out({node_out})')

            # 各ノードの線形和と（活性化関数の）出力をリストにまとめていく
            sums.append(node_sum)
            outs.append(node_out)

        # 各層内の全ノードの線形和と出力を記録
        cached_sums.append(sums)
        cached_outs.append(outs)
        next_x = outs  # 現在の層の出力（outs）＝次の層への入力（next_x）

    if cache_mode:
        return (cached_outs[-1], cached_outs, cached_sums)

    return cached_outs[-1]


# 訓練時の（1）順伝播の実行例
y_pred, cached_outs, cached_sums = forward_prop(*model, x, cache_mode=True)
# ※先ほど作成したモデルと訓練データを引数で受け取るよう改変した

print(f'cached_outs={cached_outs}')
print(f'cached_sums={cached_sums}')
# 出力例：
# cached_outs=[[0.05, 0.1], [0.5, 0.5, 0.5], [0.0]]  # 入力層／隠れ層1／出力層
# cached_sums=[[0.0, 0.0, 0.0], [0.0]]  # 隠れ層1／出力層（※入力層はない）

数値が**0.0**ばかりなので、別の計算パターンのコードも入れておきました。

In [ ]:
# 記事にはないが、別の計算パターンでもチェックしてみよう
x3 = [0.05, 0.1]
layers3 = [2, 2, 2]
weights3 = [
    [[0.15, 0.2], [0.25, 0.3]],
    [[0.4, 0.45], [0.5,0.55]]
]
biases3 = [[0.35, 0.35], [0.6, 0.6]]
model3 = (layers3, weights3, biases3)

y_pred3, cached_outs3, cached_sums3 = forward_prop(*model3, x3, cache_mode=True)
print(f'y_pred={y_pred3}')
print(f'cached_outs={cached_outs3}')
print(f'cached_sums={cached_sums3}')
# y_pred=[1.10590596705977, 1.2249214040964653]
# cached_outs=[[0.05, 0.1], [0.5932699921071872, 0.596884378259767], [1.10590596705977, 1.2249214040964653]]
# cached_sums=[[0.3775, 0.39249999999999996], [1.10590596705977, 1.2249214040964653]]

ここまでのコード中に仕込んでいる`print()`関数（※全てコメントアウトしています）のコメントを解除すると、以下のように途中の計算内容が順番にテキスト出力されます。計算内容の検証用の機能です。

![別の計算パターンの出力例](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/01-forward-prop/images/notebook-01.png)

### ●順伝播による予測の実行例

非常にシンプルで原始的な実装ですが、このように任意の層数とノード数の全結合のDNN（Deep Neural Network）のアーキテクチャーを定義して、DNNモデルによる予測が行えます。

In [ ]:
# 異なるDNNアーキテクチャーを定義してみる
layers2 = [
    2,  # 入力層の入力（特徴量）の数
    3,  # 隠れ層1のノード（ニューロン）の数
    2,  # 隠れ層2のノード（ニューロン）の数
    1]  # 出力層のノードの数

# 重みとバイアスの初期値
weights2 = [
    [[-0.2, 0.4], [-0.4, -0.5], [-0.4, -0.5]], # 入力層→隠れ層1
    [[-0.2, 0.4, 0.9], [-0.4, -0.5, -0.2]], # 隠れ層1→隠れ層2
    [[-0.5, 1.0]] # 隠れ層2→出力層
]
biases2 = [
    [0.1, -0.1, 0.1],  # 隠れ層1
    [0.2, -0.2],  # 隠れ層2
    [0.3]  # 出力層
]

# モデルを定義
model2 = (layers2, weights2, biases2)

# 仮の訓練データ（1件分）を準備
x2 = [2.3, 1.5]  # x_1とx_2の2つの特徴量

# 予測時の（1）順伝播の実行例
y_pred = forward_prop(*model2, x2)
print(y_pred)  # 予測値
# 出力例：
# [0.3828840428423274]

### ●今後のステップの準備：関数への仮引数の追加

In [ ]:
def back_prop(layers, weights, biases, y_true, cached_outs, cached_sums):
    " 逆伝播を行う関数。"
    return None, None

def update_params(layers, weights, biases, grads_w, grads_b, lr=0.1):
    " パラメーター（重みとバイアス）を更新する関数。"
    return None, None

## ■ステップ2. 逆伝播の実装

### ●逆伝播の目的と全体像

逆伝播の目的は、誤差（厳密には予測値に関する損失関数の偏微分係数）などの数値（本ノートブックでは**誤差情報**と呼ぶ）をニューラルネットに逆方向で流すこと（＝逆伝播）によって「**重みとバイアスの勾配を計算すること**」です（下の図）。

![図5　「逆伝播の流れ」のイメージ（左：ネットワーク図、右：対応する処理／数学計算）](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/02-back-prop/images/05.png)

順伝播（`forward_prop()`関数）では、計算途中に出た計算結果である「予測値（`y_pred`）」や「各ノードでの活性化関数の出力値（`cached_outs`）」と「線形和の値（`cached_sums`）」を返すだけでした。

逆伝播（`back_prop()関数`）では、計算途中に出た計算結果である「各ノードへの入力の勾配（＝逆伝播していく誤差情報）」だけでなく、「各重みの勾配（`grads_w`）」「各バイアスの勾配（`grads_b`）」の計算も必要です（下の図）。

![図6　逆伝播では各ノードへの入力／各重み／各バイアスの勾配を計算する](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/02-back-prop/images/06.png)

逆伝播では、$x_1$／$w_1$／$x_2$／$w_2$／……／$x_n$／$w_n$／$b$という大量の変数に関して、損失関数の偏微分係数（＝勾配）を計算する必要があります。

### ●損失関数：二乗和誤差

損失関数として、最も基礎的な二乗和誤差（SSE：Sum of Squared Error）を使うことにします。導関数も実装しておきます。

In [ ]:
def sseloss(y_pred, y_true):
    """
    二乗和誤差（Sum of Squared Error）の関数。
    - 引数：
    y_pred： モデルの最終出力値（output）＝予測値（prediction）。
    y_true： 目的となる値（target）＝正解値（truth、label）。
    - 戻り値：
    二乗和誤差の計算結果をfloat値で返す。
    """
    return 0.5 * (y_pred - y_true) ** 2

In [ ]:
def sseloss_der(y_pred, y_true):
    """
    二乗和誤差（Sum of Squared Error）の偏導関数。予測値（y_pred）に関して二乗和誤差関数（sseloss()）を偏微分する。
    - 引数：
    y_pred： モデルの最終出力値（output）＝予測値（predicted value）。
    y_true： 目的となる値（target）＝正解値（true/actual value、label）。
    - 戻り値：
    二乗和誤差の偏微分の計算結果（偏微分係数）をfloat値で返す。
    """

    return y_pred - y_true

偏導関数の式`y_pred - y_true`は、予測値と正解値の「誤差（Error、ズレ）」となっています。

**誤差逆伝播法**（error backpropagation）とは、この「誤差」の数値（厳密には、予測値に関しての損失関数の偏微分係数）が誤差情報としてニューラルネットワークを「逆」向きに「伝播」していく過程で、本来の目的である各重みと各バイアスの勾配を求める方法です。


![図7　各ノードでの逆伝播の処理はワンパターン](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/02-back-prop/images/07.png)

### ●1つのノードにおける逆伝播の処理

`損失関数( 活性化関数( 線形和関数( 入力、重み、バイアス ) ) )`という入れ子の関数は数学で**合成関数**と呼ばれます。

合成関数を微分するときの公式が**連鎖律**です。連鎖律を使うと、まるでマジックのように各関数の偏微分係数の掛け算だけの式に変化します（下の図）。

![図8　連鎖律を使うと各関数の偏微分の掛け算になる（各重みに関して損失関数を偏微分する例）](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/02-back-prop/images/08.png)

上の図は各重みに関して損失関数を偏微分する例ですが、各バイアスや各入力に関して損失関数を偏微分する際も連鎖律の形はほぼ同じです（下の図）。ただし入力については、前の層のノードごとに、今の層からの全てのエッジから来る各誤差情報（偏微分係数）を合計する必要があるので注意してください。

![図9　各重み／バイアス／入力に関して損失関数を偏微分する場合の連鎖律の形）](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/02-back-prop/images/09.png)

各層における各ノードの計算は、

　　「逆伝播していく誤差情報」×「活性化関数の偏微分」×「線形和関数の偏微分」

という掛け算に共通化できます（下の図）。

![図10　各層の各ノードでの計算パターンは共通化できる（出力層や隠れ層で入力の勾配を計算する例）](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/02-back-prop/images/10.png)

出力層から隠れ層まで全て、以下の4工程のワンパターンで実装できます（下の図）。

1. 逆伝播していく誤差情報
2. 活性化関数を偏微分
3. 線形和を重み／バイアス／入力で偏微分
4. 各重み／バイアス／各入力の勾配を計算

![図11　「逆伝播の流れ」の実装内容](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/02-back-prop/images/11.png)

### ●（1）逆伝播していく誤差情報

In [ ]:
# 取りあえず仮で、変数を定義して、コードが実行できるようにしておく
layer_i = 2  # 2：出力層、1：隠れ層1、0：入力層
layer_max_i = 2  # 最後の層（＝出力層）のインデックス
is_output_layer = (layer_i == layer_max_i)  # 出力層か（True）、隠れ層か（False）

# 入力層／隠れ層1／出力層にある各ノードの（活性化関数の）出力値
cached_outs = [[0.05, 0.1], [0.5, 0.5, 0.5], [0.0]]
y_true = [1.0]  # 正解値
grads_x = []  # 入力の勾配
# ---ここまでは仮の実装。ここからが必要な実装---

if is_output_layer:
    # 出力層（損失関数の偏微分係数）
    back_error = []  # 逆伝播していく誤差情報
    y_pred = cached_outs[layer_i]
    for output, target in zip(y_pred, y_true):
        loss_der = sseloss_der(output, target)  # 誤差情報
        back_error.append(loss_der)
else:
    # 隠れ層（次の層への入力の偏微分係数）
    back_error = grads_x[-1]  # 最後に追加された入力の勾配

※（1）は層ごとにまとめての処理です。以下からの（2）～（4）はノードごとの処理になります。

### ●（2）活性化関数を偏微分

In [ ]:
# 取りあえず仮で、変数を定義して、コードが実行できるようにしておく
SKIP_INPUT_LAYER = 1  # 入力層を飛ばす
cached_sums = [[0.0, 0.0, 0.0], [0.0]]  # 隠れ層1／出力層（※入力層はない）
node_sum = cached_sums[layer_max_i - SKIP_INPUT_LAYER]  # 出力層
# ---ここまでは仮の実装。ここからが必要な実装---

if is_output_layer:
    # 出力層（恒等関数の微分）
    active_der = identity_der(node_sum)
else:
    # 隠れ層（シグモイド関数の微分）
    active_der = sigmoid_der(node_sum)

### ●（3）線形和を重み／バイアス／入力で偏微分

In [ ]:
# 取りあえず仮で、変数を定義して、コードが実行できるようにしておく
PREV_LAYER = 1  # 前の層を指定するため
node_i = 0  # ノード番号

# 重みとバイアスの初期値
weights = [
    [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]], # 入力層→隠れ層1
    [[0.0, 0.0, 0.0]] # 隠れ層1→出力層
]
biases = [
    [0.0, 0.0, 0.0],  # 隠れ層1
    [0.0]  # 出力層
]
# 入力層／隠れ層1／出力層にある各ノードの（活性化関数の）出力値
cached_outs = [[0.05, 0.1], [0.5, 0.5, 0.5], [0.0]]
# ---ここまでは仮の実装。ここからが必要な実装---

w = weights[layer_i - SKIP_INPUT_LAYER][node_i]
b = biases[layer_i - SKIP_INPUT_LAYER]
x = cached_outs[layer_i - PREV_LAYER]  # 前の層の出力（out）＝今の層への入力（x）
sum_der_w = sum_der(x, w, b, with_respect_to='w')
sum_der_b = sum_der(x, w, b, with_respect_to='b')
sum_der_x = sum_der(x, w, b, with_respect_to='x')

![図12　「逆伝播していく誤差情報」「活性化関数を偏微分」「線形和を偏微分」まで実装完了](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/02-back-prop/images/12.png)

### ●（4）各重み／バイアス／各入力の勾配を計算

まずは共通の計算部分であるデルタ（`delta`変数）を計算します。

In [ ]:
delta = back_error[node_i] * active_der

![図13　デルタ（delta）のイメージ](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/02-back-prop/images/13.png)

次にバイアスの勾配（`grad_b`変数）を計算します。

In [ ]:
# 取りあえず仮で、変数を定義して、コードが実行できるようにしておく
layer_grads_b = []  # 層ごとの、バイアス勾配のリスト
# ---ここまでは仮の実装。ここからが必要な実装---

# バイアスは1つだけ
grad_b = delta * sum_der_b
layer_grads_b.append(grad_b)

![図9（再掲）　各重み／バイアス／入力に関して損失関数を偏微分する場合の連鎖律の形）](https://raw.githubusercontent.com/isshiki/neural-network-by-code/main/02-back-prop/images/09.png)

最後に各重みの勾配（`grad_w`変数）と各入力の勾配（`grad_x`変数）を計算します。

In [ ]:
# 取りあえず仮で、変数を定義して、コードが実行できるようにしておく
layer_grads_w = []  # 層ごとの、重み勾配のリスト
layer_grads_x = []  # 層ごとの、入力勾配のリスト
# ---ここまでは仮の実装。ここからが必要な実装---

# 重みと入力は前の層のノードの数だけある
node_grads_w = []
for x_i, (each_dw, each_dx) in enumerate(zip(sum_der_w, sum_der_x)):
    # 重みは個別に取得する
    grad_w = delta * each_dw
    node_grads_w.append(grad_w)

    # 入力は各ノードから前のノードに接続する全ての入力を合計する
    # （※重み視点と入力視点ではエッジの並び方が違うので注意）
    grad_x = delta * each_dx
    if node_i == 0:
        # 最初に、入力の勾配を作成
        layer_grads_x.append(grad_x)
    else:
        # その後は、その入力の勾配に合計していく
        layer_grads_x[x_i] += grad_x
layer_grads_w.append(node_grads_w)

### ●逆伝播の処理全体の実装

ニューラルネットには、層があり、その中に複数のノードが存在するという構造です。  従って、

- **逆順に**各層を1つずつ処理する`for`ループと、  
  - 層の中のノードを1つずつ処理する`for`ループの2段階構造が必要で、
    - その中に「1つのノードにおける逆伝播の処理」

を記述すればよいわけです。

In [ ]:
def back_prop(layers, weights, biases, y_true, cached_outs, cached_sums):
    """
    逆伝播を行う関数。
    - 引数：
    (layers, weights, biases)： モデルを指定する。
    y_true： 正解値（出力層のノードが複数ある場合もあるのでリスト値）。
    cached_outs： 順伝播で記録した活性化関数の出力値。予測値を含む。
    cached_sums： 順伝播で記録した線形和（Σ）値。
    - 戻り値：
    重みの勾配とバイアスの勾配を返す。
    """

    # ネットワーク全体で勾配を保持するためのリスト
    grads_w =[]  # 重みの勾配
    grads_b = []  # バイアスの勾配
    grads_x = []  # 入力の勾配

    layer_count = len(layers)
    layer_max_i = layer_count-1
    SKIP_INPUT_LAYER = 1
    PREV_LAYER = 1
    rng = range(SKIP_INPUT_LAYER, layer_count)  # 入力層以外の層インデックス
    for layer_i in reversed(rng):  # 各層を逆順に処理

        is_output_layer = (layer_i == layer_max_i)
        # 層ごとで勾配を保持するためのリスト
        layer_grads_w = []
        layer_grads_b = []
        layer_grads_x = []

        # （1）逆伝播していく誤差情報
        if is_output_layer:
            # 出力層（損失関数の偏微分係数）
            back_error = []  # 逆伝播していく誤差情報
            y_pred = cached_outs[layer_i]
            for output, target in zip(y_pred, y_true):
                loss_der = sseloss_der(output, target)  # 誤差情報
                back_error.append(loss_der)
        else:
            # 隠れ層（次の層への入力の偏微分係数）
            back_error = grads_x[-1]  # 最後に追加された入力の勾配

        node_sums = cached_sums[layer_i - SKIP_INPUT_LAYER]
        for node_i, node_sum in enumerate(node_sums):  # 各ノードを処理

            # （2）活性化関数を偏微分
            if is_output_layer:
                # 出力層（恒等関数の微分）
                active_der = identity_der(node_sum)
            else:
                # 隠れ層（シグモイド関数の微分）
                active_der = sigmoid_der(node_sum)

            # （3）線形和を重み／バイアス／入力で偏微分
            w = weights[layer_i - SKIP_INPUT_LAYER][node_i]
            b = biases[layer_i - SKIP_INPUT_LAYER]
            x = cached_outs[layer_i - PREV_LAYER]  # 前の層の出力＝今の層への入力
            sum_der_w = sum_der(x, w, b, with_respect_to='w')
            sum_der_b = sum_der(x, w, b, with_respect_to='b')
            sum_der_x = sum_der(x, w, b, with_respect_to='x')

            # （4）各重み／バイアス／各入力の勾配を計算
            delta = back_error[node_i] * active_der

            # バイアスは1つだけ
            grad_b = delta * sum_der_b
            layer_grads_b.append(grad_b)

            # 重みと入力は前の層のノードの数だけある
            node_grads_w = []
            for x_i, (each_dw, each_dx) in enumerate(zip(sum_der_w, sum_der_x)):
                # 重みは個別に取得する
                grad_w = delta * each_dw
                node_grads_w.append(grad_w)

                # 入力は各ノードから前のノードに接続する全ての入力を合計する
                # （※重み視点と入力視点ではエッジの並び方が違うので注意）
                grad_x = delta * each_dx
                if node_i == 0:
                    # 最初に、入力の勾配を作成
                    layer_grads_x.append(grad_x)
                else:
                    # その後は、その入力の勾配に合計していく
                    layer_grads_x[x_i] += grad_x
            layer_grads_w.append(node_grads_w)

        # 層ごとの勾配を、ネットワーク全体用のリストに格納
        grads_w.append(layer_grads_w)
        grads_b.append(layer_grads_b)
        grads_x.append(layer_grads_x)

    # 保持しておいた各勾配（※逆順で追加したので反転が必要）を戻り値で返す
    grads_w.reverse()
    grads_b.reverse()
    return (grads_w, grads_b)  # grads_xは最適化で不要なので返していない

### ●逆伝播の実行例

In [ ]:
x = [0.05, 0.1]
layers = [2, 2, 2]
weights = [
    [[0.15, 0.2], [0.25, 0.3]],
    [[0.4, 0.45], [0.5,0.55]]
]
biases = [[0.35, 0.35], [0.6, 0.6]]
model = (layers, weights, biases)
y_true = [0.01, 0.99]

# （1）順伝播の実行例
y_pred, cached_outs, cached_sums = forward_prop(*model, x, cache_mode=True)
print(f'y_pred={y_pred}')
print(f'cached_outs={cached_outs}')
print(f'cached_sums={cached_sums}')
# 出力例：
# y_pred=[1.10590596705977, 1.2249214040964653]
# cached_outs=[[0.05, 0.1], [0.5932699921071872, 0.596884378259767], [1.10590596705977, 1.2249214040964653]]
# cached_sums=[[0.3775, 0.39249999999999996], [1.10590596705977, 1.2249214040964653]]

# （2）逆伝播の実行例
grads_w, grads_b = back_prop(*model, y_true, cached_outs, cached_sums)
print(f'grads_w={grads_w}')
print(f'grads_b={grads_b}')
# 出力例：
# grads_w=[[[0.006706025259285303, 0.013412050518570607], [0.007487461943833829, 0.014974923887667657]], [[0.6501681244277691, 0.6541291517796395], [0.13937181955411934, 0.1402209162240302]]]
# grads_b=[[0.13412050518570606, 0.14974923887667657], [1.09590596705977, 0.23492140409646534]]

# お疲れさまでした。第1回～第2回は以上です。
第3回もこのノートブックをアップデートする予定です。